In [26]:
pip install psycopg

Note: you may need to restart the kernel to use updated packages.


In [27]:
import sqlite3
import numpy as np
import pandas as pd
import psycopg2

In [25]:
conn = sqlite3.connect('PRA1.db')

In [26]:
cursor = conn.cursor()

In [33]:
with open('BBDD_ERP_data.sql', 'r') as s:
    script = s.read()
    cursor.executescript(script)

In [41]:
conn.commit()

In [42]:
#VER TABLAS DE LA BD

q=""" 
 SELECT * FROM sqlite_master WHERE type = "table";
 """
df = pd.read_sql(q, conn)
df

,type,name,tbl_name,rootpage,sql
0,table,tb_company,tb_company,2,CREATE TABLE tb_company\n (\n ...
1,table,tb_customer,tb_customer,4,CREATE TABLE tb_customer\n (\n ...
2,table,tb_site,tb_site,6,CREATE TABLE tb_site\n (\n c...
3,table,tb_iva,tb_iva,12,CREATE TABLE tb_iva\n (\n co...
4,table,tb_invoice,tb_invoice,8,CREATE TABLE tb_invoice\n (\n ...
5,table,tb_lines,tb_lines,10,CREATE TABLE tb_lines\n (\n ...


* a) Proporcionar los nombres de los clientes que empiezan por AGR.  
    Queremos obtener el número de cliente, nombre y NIF.  
    Los datos deben aparecer ordenados por número de cliente en orden descendiente.


In [40]:
q=""" 
SELECT cust_no, cust_name, cust_cif
FROM tb_customer
WHERE cust_name LIKE 'AGR%'
ORDER BY cust_no DESC
"""
df = pd.read_sql(q, conn)
df

,cust_no,cust_name,cust_cif
0,C0016,AGR SANT JOAN,B81232171
1,C0007,AGRIMARCOS,B98766562
2,C0006,AGR AGRI,B78788999
3,C0002,AGRICULTURA VIVES,A66666666


* b) Proporcionar las facturas de clientes con dirección en la ciudad de Granada.  
    Queremos obtener el número de cliente, nombre de cliente, dirección, ciudad y número de factura.  
    La consulta debe estar ordenada por número de cliente ascendientemente y número de factura descendentemente.

In [65]:
q=""" 
SELECT i.cust_no, c.cust_name, s.site_address, s.site_city, i.invoice_no
FROM tb_invoice as i
INNER JOIN tb_customer as c ON i.cust_no=c.cust_no
INNER JOIN tb_site as s ON c.cust_no=s.cust_no
WHERE s.site_city LIKE "GRANADA"
"""
df = pd.read_sql(q, conn)
df.pivot_table(index=['cust_no', 'cust_name','site_address', 'site_city', 'invoice_no'])

Empty DataFrame
Columns: []
Index: [(C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000001), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000002), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000003), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000004), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000005), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000006), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000007), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000008), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000009), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000010), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000011), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000012), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000013), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000014), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000015), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000016), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000017), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000018), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000019), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000020), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000021), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000022), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000023), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000024)]

* c) Proporcionar la/s compañía/s con mayor número de facturas.  
    Queremos obtener el código de compañía, el nombre de la compañía y el número de facturas.  
    En caso de haber más de una compañía, los resultados deben aparecer ordenados por nombre de compañía.